# Example notebook: Search data with CasJobs

SciServer Compute can talk to other components of SciServer through a series of <em>modules</em>, one for each component. This example notebook shows how to use the <strong><code>SciServer.CasJobs</code></strong> module to search for data within your Python scripts. CasJobs is SciServer's data search tool, allowing you to select data from any of our big data datasets and/or your own uploaded datasets.

You are welcome (encouraged!) to copy these examples into another folder and modify them to meet your needs. You can use them as a starting point to create your own scripts. Please do not edit this notebook directly, because your edits may be overwritten if changes to the SciServer modules require changes to these example notebooks.

To run the example Python scripts in this notebook, click in any of the Code cells below (the ones with the gray backgrounds). Click the play button at the top of the window (just below the menubar) to run the script, or press Shift-Enter. When you run a cell, its output of will appear directly below the cell.

## Import modules

Like any Python modules, the SciServer modules must be imported before being used. The next code block first imports the SciServer modules you will need for this example notebook, then imports some other required modules. Comments in the code block explain what each module does. To learn how to import other modules, see the Python 3.5 import documentation (https://docs.python.org/3.5/reference/import.html), or the documentation of the module(s) you are trying to import.

In [1]:
import SciServer
from SciServer import CasJobs     # Communicate between SciServer Compute and CasJobs
print('Imported SciServer modules')

import pandas                                # data analysis tools
import numpy as np                           # numerical tools
from datetime import datetime, timedelta     # date and timestamp tools
from pprint import pprint                    # print human-readable output
print('Imported other needed modules')

Imported SciServer modules
Imported other needed modules


## Get help

At any point after the modules are imported, you can type "help (<em>name of module</em>)" to read the documentation for that module. This is true for all SciServer modules and most other modules as well. Try it below.

In [ ]:
# Read the help document for the entire SciServer package
help(SciServer)

In [ ]:
# Read the help document for the CasJobs module
help(CasJobs)

### Optional: create functions

These are convenience functions.

In [16]:
# These convenience functions will print your list of tables in human-readable format

def better_createdate(cjCreateDate):     # returns table create date as a datetime object
    createsec = cjCreateDate / 10000000  # Divide by 10 million to get seconds elapsed since 1 AD
    firstday = datetime(1, 1, 1, 0, 0)   # Save 1 AD as "firstday"
    created = firstday + timedelta(seconds=createsec)  # Get calendar date on which table was created
    return (created)

def tables_formatted(tables):   # better formatted printing of a tables dictionary (output of get_tables)
# Returns the following information abaout the tables in your MyDB (as a Python dictionary object):
### Size: size of the table (in kB)
### Name: the name of the table
### Rows: the number of rows the table contains
### Date: the date of the table's creation, as the number of 10-microsecond intervals elapsed 1 AD

    tables = sorted(tables, key=lambda k: k['Name']) # alphabetize by table name
    for thistable in tables:
        print('Table name:\t',thistable['Name'])
        print('Rows:\t\t {:,.0f}'.format(thistable['Rows']))
        print('Size (kB):\t {:,.0f} '.format(thistable['Size']))
        betterdate = better_createdate(thistable['Date'])
        print('Created time:\t',betterdate.strftime('%Y-%m-%d %H:%M:%S'))
        print('\n')
        
#submit a job, which inserts the query results into a table in the MyDB database context. 
#Wait until the job is done and get its status.

def translate_status(status):
    if (status == 0):
        status_word = 'Ready'
    elif (status == 1):
        status_word = 'Started'
    elif (status == 2):
        status_word = 'Cancelling'
    elif (status == 3):
        status_word = 'Cancelled'
    elif (status == 4):
        status_word = 'Failed'
    elif (status == 5):
        status_word = 'Finished'
    else:
        status_word = 'Status not found!!!!!!!!!'
    return (status_word)

def jobDescriber(jobDescription):
    print('JobID: ',jobDescription['JobID'])
    print('Status: ',translate_status(jobDescription["Status"]),' (',jobDescription["Status"],')')
    print('Message: ',jobDescription['Message'])
    print('Created_Table: ',jobDescription['Created_Table'])
    print('Rows: ',jobDescription['Rows'])
    wait = pandas.to_datetime(jobDescription['TimeStart']) - pandas.to_datetime(jobDescription['TimeSubmit'])
    duration = pandas.to_datetime(jobDescription['TimeEnd']) - pandas.to_datetime(jobDescription['TimeStart'])
    print('Wait time: ',wait.seconds,' seconds')
    print('Query duration: ',duration.seconds, 'seconds')
        
print('Created functions')

Created functions


## What data can I search?

CasJobs allows you to search many different datasets, referred to as <strong>contexts</strong> (they are known as contexts so they can be described independently of the databases in which they are stored). Each context consists of one or more tables containing data or metadata related to a single aspect of the full dataset.

### Get a list of contexts

At the moment, the SciServer.CasJobs module does not have a function to list available contexts. The best way to see what contexts are available to you is to log in to <a href="http://skyserver.sdss.org/casjobs/" target="_blank">CasJobs</a> (link opens in a new window). Once you are logged in, you should see the Query page. Look for the <strong>Contexts</strong> dropdown menu toward the top left of the page, just above the big textbox. The values in that dropdown list show the contexts you can search, both directly in CasJobs and in Compute.

### Show data tables in a context

Once you know what context you want to search, you can use the <strong>CasJobs.getTables(context)</strong> function to show the data tables in that context. The Code cell below gives commands to list all tables in a context. Set the value of <em>this_context</em> to be the context you want to see. The function CasJobs.getTables(context) returns a list of Python dictionaries, one dictionary per table.

Each dictionary in the list contains the following information about one table:
<ul>
<li><em>Date:</em> the number of 10-millisecond intervals since the table was created</li>
<li><em>Name:</em> the name of the table</li>
<li><em>Rows:</em> the number of rows in the table</li>
<li><em>Size:</em> the size of the table in kilobytes</li>
</ul>

The code cell gives two options for printing the list of tables: using Python's pprint library or using the tables_formatted(tableList) convenience function defined above. The convenience function sorts the list of tables alphabetically by name, and displays the dates into datetime values. Try uncommenting and commenting those lines to see both options.

In [3]:
this_context = "MyDB"    # Your MyDB
#this_context = 'dr14'   # SDSS Data Release 14

tables = CasJobs.getTables(context=this_context)
print('Tables in '+this_context+':\n')


#pprint(tables)   # Standard human-readable printing using Python's pprint module
tables_formatted(tables)  # Sorting and better printing using a convenience function

Tables in MyDB:

Table name:	 GalaxyThumbs
Rows:		 16
Size (kB):	 16 
Created time:	 2017-06-21 13:01:20


Table name:	 GalaxyThumbs2
Rows:		 48
Size (kB):	 16 
Created time:	 2017-06-16 08:45:05


Table name:	 GalaxyThumbs3
Rows:		 16
Size (kB):	 16 
Created time:	 2017-06-16 10:43:58


Table name:	 MyNewtable22
Rows:		 1
Size (kB):	 16 
Created time:	 2017-08-02 15:30:43


Table name:	 MyNewtable55
Rows:		 1
Size (kB):	 16 
Created time:	 2017-08-02 16:23:49


Table name:	 MyTable
Rows:		 0
Size (kB):	 0 
Created time:	 2017-09-20 15:40:23


Table name:	 QuickResults
Rows:		 1
Size (kB):	 80 
Created time:	 2017-09-13 12:31:38


Table name:	 SkyServer_Book
Rows:		 2
Size (kB):	 16 
Created time:	 2013-06-19 14:32:14


Table name:	 apogeesegue
Rows:		 794
Size (kB):	 64 
Created time:	 2013-07-18 10:57:09


Table name:	 boss_martin
Rows:		 2,332,836
Size (kB):	 66,952 
Created time:	 2017-05-23 08:47:02


Table name:	 danadr12withgz
Rows:		 27,868
Size (kB):	 2,312 
Created time:	 201

## Run a quick query and get results directly

Now that you know what contexts (datasets) are available to you, and you know what tables can be found in those contexts, you are ready to write and submit an SQL query to that context. A query is a request for data, written in SQL (Structured Query Language), a programming language designed for efficient database searches. 

SkyServer features a <a href="http://skyserver.sdss.org/dr14/en/help/howto/search/searchhowtohome.aspx" target="_blank">tutorial for learning SQL</a>, as well as <a href="http://skyserver.sdss.org/dr14/en/help/docs/sql_help.aspx" target="_blank">tips for writing good queries</a> and a long list of <a href="http://skyserver.sdss.org/dr14/en/help/docs/realquery.aspx" target="_blank">sample queries</a> that you can adapt to create your queries.

Once you have a query, you can get results by running (executing) it in CasJobs. To do this from a script in SciServer Compute, use the <strong><code>CasJobs.executeQuery(sql,...)</code></strong> function. The function takes as input an SQL query (and optional parameters listed below), and returns a table containing query results (in one of several formats with a default of a <a href="https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html" target="_blank">pandas dataframe</a>.

The <em>sql</em> paramter is required. The <em>context</em> paramter is recommended to explicitly state the context to which the query will be submitted; the default value is 'MyDB'. For a full list of parameters taken by the CasJobs.executeQuery(sql, ...) function, see <a href="http://www.sciserver.org/docs/sciscript-python/SciServer.html#module-SciServer.CasJobs" target="_blank">its documentation on the SciServer documentation site</a>.

### Example

The Code cell below gives an example of a query that will run quickly in CasJobs and return 10 results. It also includes (commented out) examples of an invalid SQL statement and a valid statement that returns no results, so you can see what happens in those cases.

In [ ]:
# Example of the function CasJobs.executeQuery(sql, ...)
#   This function executes a quick SQL query and store results in a pandas dataframe.

# A valid query that returns results.
### Return IDs, positions, and g magnitudes (brightness measurements in red wavelength of light)...
#### for 10 galaxies with reliable imaging data.

myquery = 'select top 10 objid, ra, dec, r ' # note space after select block
myquery += 'from galaxy '
myquery += 'where clean = 1'

# A badly-formed query
#badquery = "ceci n''est pas une query"   # note subtitution of '' for single quote mark

# A valid query that returns no results
#zeroquery = "select top 10 * from specobj where 0=1"

# Execute query, return a pandas dataframe, then show it
df = CasJobs.executeQuery(sql=myquery, context="DR14")
df

## Run a longer query and store results in your MyDB

The above example shows a quick query. Quick queries are limited to 90 seconds of processing time and <strong><em>XXX results (rows)</em></strong> returned. For longer queries, CasJobs has a system of <strong>jobs</strong>. When you submit a query to CasJobs, the system creates a job for your query, with a unique <code>jobId</code>. CasJobs then runs this job in the background as server resources permit. When it completes, the job writes the query results into your MyDB personal database space, into a table that you can specify. You can later retrieve and use the query results associated with the job by querying that table in your MyDB (that is, by executing a query with <code>context='MyDB'</code>.

To run a long query with MyDB output from SciServer Compute, you would use a different approach from the one you used in the last section. Submit the query using the function <strong><code>CasJobs.submitJob(sql,context)</code></strong>. The context used in this function is the context whose data you want to search (since the data will automatically be written into the <code>MyDB</code> context).

Importantly, the function <strong><code>CasJobs.submitJob(sql,context)</code></strong> does NOT return query results directly. It returns only an integer <code>jobID</code> of the job that corresponds to your query. To retrieve the data and use it in a Compute script, you must still use <strong><code>CasJobs.executeQuery(sql,context='MyDB',...)</code></strong> to retrieve results and store them in memory (e.g. into a pandas dataframe), as described in the previous section.


Once you have a query, you can get results by running (executing) it in CasJobs. To do this from a script in SciServer Compute, use the <strong>CasJobs.executeQuery(sql, ...)</strong> function. The function takes as input an SQL query (and optional parameters listed below), and returns a table containing query results (in one of several formats with a default of a <a href="https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html" target="_blank">pandas dataframe</a>.

The <em>sql</em> paramter is required. The <em>context</em> paramter is recommended to explicitly state the context to which the query will be submitted; the default value is 'MyDB'. For a full list of parameters taken by the CasJobs.executeQuery(sql, ...) function, see <a href="http://www.sciserver.org/docs/sciscript-python/SciServer.html#module-SciServer.CasJobs" target="_blank">its documentation on the SciServer documentation site</a>.

### Check on running jobs



### Example

The Code cell below gives an example of a query that will run quickly in CasJobs and return 10 results. It also includes (commented out) examples of an invalid SQL statement and a valid statement that returns no results, so you can see what happens in those cases.

In [18]:
bigtablename = 'hugetable3'

verylongquery = 'select top 1000000 objid, ra, dec \n'
verylongquery += 'u, g, r, i, z, err_u, err_g, err_r, err_i, err_z, petror90_r \n'
verylongquery += 'into mydb.' + bigtablename + '\n'
verylongquery += 'from galaxy\n'
verylongquery += 'where clean = 1'

print('Submitting query:\n',verylongquery)
print('\n')

jobId = CasJobs.submitJob(sql=verylongquery, context="DR14")

print('Job submitted with jobId = ',jobId)
print('\n')

jobDescription = CasJobs.waitForJob(jobId=jobId)#, verbose=True)
print('\n')

print('Information about the job:')
jobDescriber(jobDescription)
#pprint(jobDescription)



#print('\n')
#print('Using quick query return error "Query exceeds queue time":')
#df = CasJobs.executeQuery(sql=verylongquery, context="DR14")



Submitting query:
 select top 1000000 objid, ra, dec 
u, g, r, i, z, err_u, err_g, err_r, err_i, err_z, petror90_r 
into mydb.hugetable3
from galaxy
where clean = 1


Job submitted with jobId =  32160394


Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Done!


Information about the job:
JobID:  32160394
Status:  Finished  ( 5 )
Message:  Query Complete
Created_Table:  hugetable3
Rows:  1000000
Wait time:  1  seconds
Query duration:  75 seconds


## Thank you!

Thanks for reviewing this SciServer example notebook. You can use this notebook as a template to develop your own notebooks, but please do so in a copy rather than in the original example notebook.
As you begin to use any of our SciServer modules in your own notebooks, consult the SciServer scripting documentation at http://www.sciserver.org/docs/sciscript-python/SciServer.html.

If you have questions, please email the SciServer helpdesk at sciserver-helpdesk@jhu.edu .

In [8]:
#get user schema info

casJobsId = CasJobs.getSchemaName()
print(casJobsId)

wsid_2133771900


In [ ]:
# Define some very simple test cases for the purposes of this example notebook

CasJobs_TestDatabase = "MyDB" # use your MyDB - normally you won't need to change this
CasJobs_TestQuery = "select 4 as Column1, 5 as Column2 " # toy query, simply returns two columns of constants: 4, 5.
CasJobs_TestTableName1 = "MyNewtable1"    # the name of a new table to be created (doesn't yet exist)
CasJobs_TestTableName2 = "MyNewtable37"    # the name of a new table to be created (doesn't yet exist)
CasJobs_TestTableCSV = u"Column1,Column2\n6,7\n"    # toy CSV file: 1 row x 2 columns of constants: 6, 7.
CasJobs_TestCSVFile = "SciScriptTestFile.csv"       # a more real CSV file. It must be included in this dir in current container.
CasJobs_TestFitsFile = "SciScriptTestFile.fits"     # a FITS file to upload

print('Test case variables set.')